#2022년 지역별 사망 교통사고 데이터 분석 및 시각화


*   어느 지역이 가장 많은 사망 교통사고기 발생 했었나
*   어느 시간대에 가장 많은 교통사고가 발생하였는가
* 교통사고 위치와 우리집 위치 거리는 얼마나 될까?
* 우리집 위치와 가까운 곳에 교통사고가 발생 했었나?



##01.데이터 준비하기

* https://www.data.go.kr/data/15070340/fileData.do
* 도로교통공단에서 제공하는 사망 교통사고 데이터

###패키지 설치

In [ ]:
!pip install koreanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.8 MB/s eta 0:00:00


###패키지 임포트

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import koreanize_matplotlib

import folium #지도 시각화 패키지

import warnings
warnings.filterwarnings(action='ignore')

###데이터 불러오기

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ABC/22년 사망 교통사고자료.csv', encoding='euc-kr')
df

###데이터 확인하기


In [ ]:
df.info() # 데이터프레임이 어떤 구조로 되어 있는지
#인덱스-컬럼개수-결측치들이 있는지-데이터타입 확인

##02.사망 교통사고 데이터 전처리

### 원본데이터 복사

In [ ]:
accident_df =df.copy()
accident_df.head()

###발생년월일시 컬럼에서 시간 파생 컬럼 만들기
* 어느지역, 시간대를 알기 위해 합쳐진 데이터에서 시간만 나누기

In [ ]:
accident_df.info()

In [ ]:
#1)문자열 슬라이싱 : 문자열로 타입 변경 (int 형의 컬럼을 string형 변환)
accident_df=accident_df.astype({'발생년월일시':'string'}) #딕셔너리 구조
accident_df.info()

In [ ]:
#2) 시간으로 분리한 데이터를 발생기간 컬럼에 담기
accident_df['발생시간'] = accident_df['발생년월일시'].str[-6:-3]
accident_df.head(30)

In [ ]:
#3)발생시간 컬럼을 숫자 데이터 타입으로 변경
accident_df=accident_df.astype({'발생시간':'int'}) #딕셔너리 구조
accident_df.info()

###발생년월일시 컬럼에서 발생년월일 파생 컬럼 만들기

In [ ]:
#발생년월일시(string)-> 발생년월일(날짜타입, datetime타입으로 변경)
accident_df['발생년월일'] = pd.to_datetime(accident_df['발생년월일시'].str[:-6],format='%Y-%m-%d',errors='raise')
accident_df.info()
accident_df['발생년월일']

###필요없는 컬럼 삭제 및 컬럼 정리

In [ ]:
#1)필요없는 컬러 삭제
del accident_df['발생년월일시']
accident_df.info()

In [ ]:
#2) 컬럼 순서 정리 - 발생년, 발생년월일, 발생시간 , ...


#2-1)컬럼 리스트 정제 - 컬럼명을 리스트로 만들기
col1=accident_df.columns[-2:].tolist()
col1.reverse() #reverse()순서 바꾸기
col1

col2 =accident_df.columns[1:-2].tolist()

col_list=col1+col2
col_list.insert(0,'발생년')

col_list

In [ ]:
#2-2)원하는 컬럼의 순서대로 추출 후 다시 담기

accident_df=accident_df[col_list]
accident_df

###정제된 데이터셋 저장

In [ ]:
accident_df.to_csv('2022년 사망 교통사고 전처리_20240704.csv',
                    encoding='utf-8-sig',index=False)

##03.2019년 사망 교통사고 데이터 시각화

###날짜별/시간별 교통사고 현황

In [ ]:
fig =px.scatter(accident_df,x='발생년월일',y='발생시간',
                size='사망자수', color='발생지시도') #fig는 데이터 전체를 주고 x,y축만 명명해주면 돼서 편함. scatter
fig.show()

### 시간대별 교통사고 사망자 현황


In [ ]:
fig =px.bar(accident_df, x='발생시간', y='사망자수')
fig.show()

In [ ]:
fig =px.bar(accident_df, x='사망자수', y='발생시간',orientation='h') #orientation='h'
fig.show()

### 시도별 교통사고 사망자 현황


In [ ]:
accident_df['발생지시도'].unique() #데이터마다 지시도가 다르게 입력되어 있을 수 있어 데이터 확인을 해야 됨.
location_list = accident_df['발생지시도'].unique()
location_list.sort()
location_list

In [ ]:
fig =px.bar(accident_df, x='사망자수', y='발생지시도', category_orders={'발생지시도':location_list})
fig.show()

In [ ]:
fig =px.histogram(accident_df, x='사망자수', y='발생지시도',
                  orientation='h',text_auto=True, category_orders={'발생지시도':location_list})
fig.show()

In [ ]:
fig =px.histogram(accident_df, x='사망자수', y='발생지시도',
                  orientation='h',text_auto=True,
                  category_orders={'발생지시도':accident_df['발생지시도'].value_counts().index })
fig.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=accident_df, x='발생지시도',
              order= location_list)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(data=accident_df, x='발생지시도',
              order= accident_df['발생지시도'].value_counts().index)
plt.show()


### 요일별로 교통사고 현황
* 어느 요일이 교통사고가 가장 많이 발생했는지
* 월 - 일 순으로 x축 정렬

In [ ]:
day_list=['월','화','수','목','금','토','일']
day_list


In [ ]:
fig =px.histogram(accident_df, x='사망자수', y='요일',
                  orientation='h',text_auto=True,
                  category_orders={'요일': day_list }) #사망자수
fig.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=accident_df, x='요일',
              order= accident_df['요일'].value_counts().index) #value_counts()은 사건 발생수 세부적인 사망자 수는 모름
plt.show()

### 사고 유형별 교통사고 현황

In [ ]:
fig =px.histogram(accident_df, x='사망자수', y='사고유형',
                  orientation='h',text_auto=True,
                  category_orders={'사고유형': day_list }) #사고유형을 확인해보면 횡단중 사고가 가장 많은 걸로 보아 피해자는 대부분 일반 보행자
fig.show()

In [ ]:
fig =px.histogram(accident_df, x='사망자수', y='가해자법규위반',
                  orientation='h',text_auto=True,
                  category_orders={'가해자법규위반': day_list })
fig.show()


##4.경기도 사망 교통사고 데이터 분석 및 시각화

###경기도 데이터 추출

In [ ]:
gg_df = accident_df.query("발생지시도=='경기'")
gg_df.info()

###발생시간 별 교통사고 사망자 현황

In [ ]:
#전국은 19시에 가장 많은 교통사고 발생 -> 대전은 20시에 발생
fig =px.bar(gg_df, x='사망자수', y='발생시간', orientation='h')
fig.show()

### 지역구별 교통사고 사망자 현황

In [ ]:
fig =px.bar(gg_df, x='사망자수', y='발생지시군구', orientation='h')
fig.show()

###월별 교통사고 사망자 현황

In [ ]:
accident_df ['발생월']= accident_df['발생년월일'].dt.month
gg_df['발생월']=gg_df['발생년월일'].dt.month
gg_df.head()

In [ ]:
#월별 교통사고를 전국 vs 대전
fig, axes = plt.subplots(1,2,figsize=(15,5))
plt.suptitle('2022년 월별 교통사고 발생건수 비교')

sns.countplot(data=accident_df, x='발생월', ax=axes[0],
              order=accident_df['발생월'].value_counts().index)

axes[0].set_title('전국')

sns.countplot(data=gg_df, x='발생월', ax=axes[1],
              order=gg_df['발생월'].value_counts().index)

axes[1].set_title('경기도')

#비교하는 차트의 y축 고정하기
plt.ylim(0,accident_df['발생월'].value_counts().max()) #accident_df['발생월'].value_counts().max() 전국의 발생건수의 최대값으로 기준

plt.show()

##05. 지도 시각화

In [ ]:
map =folium.Map(location=[37.567167, 127.190292]) #location 중심좌표(좌표) 넣기
map

In [ ]:
gg_df=gg_df.astype({'사망자수':'float','부상자수':'float'})
gg_df.info()

In [ ]:
#1)지도 준비
map =folium.Map(location=[37.567167, 127.190292]) #location 중심좌표(좌표) 넣기

#2)CircleMarker 지도에표출 -> for문을 이용해서 dj_df에 정보가 있을 때 까지 지도에 추가 add
for n in gg_df.index:
  #CircleMarker의 사이즈
  cnt =gg_df['사망자수'][n] + gg_df['부상자수'][n]


  #사고 위치 위도,경도 추출
  lat = gg_df['위도'][n]
  lng = gg_df['경도'][n]

  #지도에 추가 add
  folium.CircleMarker(location=[lat,lng],radius=cnt,#radius은 CircleMarker 반지름
                      popup = gg_df['사고유형'][n],
                      color ='red', fill_color='red').add_to(map)
map

In [ ]:
map.save('2022년 경기도 사망 교통사고 현황.html')